# Loading and cleaning scraped data

## Brief explanation 

**English version**  
We collected the data on the website https://immobilier.lefigaro.fr/ for the 3 biggest french cities: Paris, Lyon, Marseille. Scrapping was done by a fellow student, Foudil.

The output of the scraper was a .txt file for each of the 3 cities. The outputs of the scraper were inconsistent, because some appartments were listed as houses, some houses as appartments; we also had listings of parking spaces, offices etc. We'll tackle the cleaning of the file and the setup of the dataset in this notebook. Ideally, we would create a pipeline for extracting, transforming, cleaning and loading the data, in a .py script. But for this academic/personal project we're gonna stick to a notebook.

**French version**  
Nous avons collecté des données sur le site web https://immobilier.lefigaro.fr/ pour les 3 plus grandes villes de France : Paris, Lyon et Marseille. Le scrapping a été éffectué par mon collègue, Foudil. 

Le résultat de notre collecte de données était un fichier .txt pour chacune des 3 villes. Les résultats de la collecte étaient inconsistants, car certains appartements étaient listés comme des maisons, certaines maisons comme des appartements, et nous avions également des listes de places de parking, de bureaux, etc. Je vais m'attaquer au nettoyage des fichiers et à la mise en place du dataframe/dataset dans ce notebook. Idéalement, je devrais créer un script .py, avec une pipeline pour extraire, transformer, nettoyer et charger les données. Mais pour ce projet académique/personnel, je vais rester sur un notebook.

In [1]:
from typing import Tuple, List
import pandas as pd
import os


## Defining the functions

**EN**  
Below are defined all the functions that we're going to use to clean the data. Docstrings are available for each function.  
**FR**  
Ci-dessous sont définies toutes les fonctions que nous allons utiliser pour nettoyer les données. Des docstrings sont disponibles pour chaque fonction.

In [2]:
def string_replace(string:str, str_to_replace=None, replace_with=None) -> str:
    """
    Replace the occurrences of a substring in a given string with a replacement string.

    Args:
        string (str): The input string to be processed.
        str_to_replace (str): The substring to be replaced in the input string. If not provided, defaults to None.
        replace_with (str): The replacement string that will be used to replace the occurrences of the substring. If not provided, defaults to None.

    Returns:
        str: The modified string after replacing the occurrences of the substring with the replacement string.
    """
    string = string.replace(str_to_replace, replace_with)
    return string

def file_read(file_name:str) -> List[str]:
    """ Read the file and return the lines"""
    with open(file_name) as f:
        lines = f.readlines()
    return lines

def create_copy(orig_file_name: str, orig_file_folder:str, output_folder_for_copy: str) -> None:
    """Create a copy of the original files"""
    if not os.path.exists(output_folder_for_copy):
        os.mkdir(output_folder_for_copy)
    orig_file_path = os.path.join(orig_file_folder, orig_file_name)
    copied_file_path = os.path.join('cleaned_data', os.path.basename(orig_file_path))
    with open(orig_file_path, 'r') as o_f, open(copied_file_path, 'w') as c_f:
        c_f.writelines(o_f.readlines())

def file_write(file_name:str, list_to_replace=None, list_replace_with=None)-> None:
    """
    Read a file, replace the specified substrings in its lines, and overwrite the original file.

    Args:
        file_name (str): The path to the file to be read and overwritten.
        list_to_replace (list): A list of substrings to be replaced in each line of the file. If not provided, defaults to None.
        list_replace_with (list): A list of replacement strings that will be used to replace the corresponding substrings in the file. If not provided, defaults to None.

    Returns:
        None: The function overwrites the original file and does not return a value.
    """

    lines = file_read(file_name)
    with open(file_name, 'w') as f:
        for line in lines:
            for tr, rw in zip(list_to_replace, list_replace_with):
                line = string_replace(string=line, str_to_replace=tr, replace_with=rw)
            f.write(line)


def prop_printer(file_name:str, df_name:str, n_semicolons:int) -> None:
    """
    Prints the proportion of lines in a file that contain a different number of semicolons than n_semicolons.

    Args:
    - file_name (str): The name of the file to be processed.
    - n_semicolons (int): The number of semicolons to be searched for in each line.

    Returns:
    - None
    """
    list_obj = []
    with open(file_name, 'r') as f:
        lines = f.readlines()
    for line in lines:
        if line.count(';')!=n_semicolons:
            list_obj.append(line)
    if len(lines)>0:
        print(f"Proportion of flagged lines for {df_name}: {len(list_obj)/len(lines)*100:.2f} %")
    else:
        print("The file is empty")

def val_inval_lists(file_name: str, n_semicolons: int) -> Tuple[List[str], List[str]]:
    """
    Separates the lines in a file into two lists based on the number of semicolons in each line.

    Args:
    - file_name (str): The name of the file to be processed.
    - n_semicolons (int): The number of semicolons to be searched for in each line.

    Returns:
    - A tuple of two lists of strings. The first list contains the lines that have the specified number of semicolons,
      while the second list contains the lines that do not have the specified number of semicolons.
    """
    valid_list = []
    invalid_list = []
    with open(file_name, 'r') as f:
        lines = f.readlines()
    for line in lines:
        if line.count(';')== n_semicolons:
            valid_list.append(line)
        else:
            invalid_list.append(line)
    return valid_list, invalid_list

def drop_nonconform_lines(file_name:str, n_semicolons:int)->None:
    """
    Drops lines from a file that do not have the specified number of semicolons.

    Args:
    - file_name (str): The name of the file to be processed.
    - n_semicolons (int): The number of semicolons to be searched for in each line.

    Returns:
    - None

    Raises:
    - FileNotFoundError: If the specified file is not found.
    - TypeError: If the specified file is not a string or if n_semicolons is not an integer.
    """
    with open(file_name, 'r') as f:
        lines = f.readlines()
    with open(file_name, 'w') as f:
        for line in lines:
            if line.count(';')==n_semicolons:
                f.write(line)

## Analyzing the contents of the text files
**EN**  
We're going to analyze the contents of the text files to have an idea of the missing values and non-conform lines. I decided to separate the data in 3 fields: Information about the listing, Information about the size of the listing, Price. This is done for all 3 files. Each line should contain exactly these 3 fields, that's why our function checks for 2 semicolons and separates invalid lines from valid lines. 
We then print a message about the % of non-conform lines to look further into what's causing the issue.  
**FR**  
Nous allons analyser le contenu des fichiers texte pour avoir une idée des valeurs manquantes et des lignes non conformes. J'ai décidé de séparer les données en 3 champs : Information sur l'annonce, Information sur la taille de la propriété, Prix. Cela est fait pour les 3 fichiers. Chaque ligne doit contenir exactement ces 3 champs, c'est pourquoi notre fonction vérifie la présence de 2 points-virgules et sépare les lignes invalides des lignes valides. Nous affichons ensuite un message sur le pourcentage de lignes non conformes pour examiner ce qui pose des problèmes.


In [3]:
chars_to_replace = ['ème', 'è', 'é', '€', 'm²', ' (75)', ' (69)', ' (13)', 'Terrain', 'terrain', 'er']
replace_chars_with_paris = [';', 'e', 'e', 'eur', 'm2', '', '', '', ';']
replace_chars_with = [';', 'e', 'e', 'eur', 'm2;', '', '', '', '','',';']

cwd = os.getcwd()
cwd = os.path.normpath(cwd).replace('\\', '/')
orig_path = cwd + '/original_data'
paris = orig_path + '/annuaire_paris.txt'
lyon = orig_path + '/annuaire_lyon.txt'
marseille = orig_path + '/annuaire_marseille.txt'

output_path = cwd + '/cleaned_data'
files = [paris, lyon, marseille]
for file in files:
    create_copy(orig_file_name = file, orig_file_folder = orig_path, output_folder_for_copy=output_path)

paris = output_path + '/annuaire_paris.txt'
lyon = output_path + '/annuaire_lyon.txt'
marseille = output_path + '/annuaire_marseille.txt'

file_write(paris, chars_to_replace, replace_chars_with_paris)
file_write(lyon, chars_to_replace, replace_chars_with)
file_write(marseille, chars_to_replace, replace_chars_with)

prop_printer(paris, 'Paris', 2)
prop_printer(lyon,'Lyon', 2)
prop_printer(marseille, 'Marseille', 2)

Proportion of flagged lines for Paris: 2.27 %
Proportion of flagged lines for Lyon: 5.35 %
Proportion of flagged lines for Marseille: 11.42 %


## Digging further 
**EN**   
We'll dig further into understanding why our function flagged **2.27%**, **5.35%**, **11.42%** lines of each respective file. To do so, we're looking to filter listings that contain the word `appartement` and separate them into 2 lists: *Valid* and *Invalid*. We then print the proportion of flagged appartments. (As mentioned above, our scraper output contains parking lots, office spaces, houses, etc.)  
**FR**   
Nous allons chercher à compréndre le(s) motif(s) pour lequel(s) notre fonction a signalé **2.27%**, **5.35%** et **11.42%** des lignes comme non-conformes. Pour cela, nous allons filtrer les annonces qui contiennent le mot `appartement` et les séparer en deux listes : *Valide* et *Invalide*. Nous afficherons ensuite la proportion d'appartements signalés. (Comme mentionné précédemment, la sortie de notre scraper contient des parkings, des bureaux, des maisons, etc.)


In [4]:
lyon_val, lyon_inval = val_inval_lists(lyon, 2)
lyon_appart_list = []
for i,val in enumerate(lyon_inval):
    if 'appartement' in val:
        lyon_appart_list.append(val)
print(f"The appartments that were flagged as invalid in Lyon file represent \
{100*len(lyon_appart_list)/(len(lyon_val)+len(lyon_inval)):.2f}% of the dataset")

The appartments that were flagged as invalid in Lyon file represent 1.42% of the dataset


In [5]:
marseille_val, marseille_inval = val_inval_lists(marseille, 2)
marseille_appart_list = []
for i,val in enumerate(marseille_inval):
    if 'appartement' in val:
        marseille_appart_list.append(val)
print(f"The appartments that were flagged as invalid in Marseille file represent \
{100*len(marseille_appart_list)/(len(marseille_val)+len(marseille_inval)):.2f}% of the dataset")

The appartments that were flagged as invalid in Marseille file represent 1.83% of the dataset


In [6]:
paris_val, paris_inval = val_inval_lists(paris, 2)
paris_appart_list = []
for i, val in enumerate(paris_inval):
    if 'appartement' in val:
        paris_appart_list.append(val)
print(f"The appartments that were flagged as invalid in Paris file represent \
{100*len(paris_appart_list)/(len(paris_val)+len(paris_inval)):.2f}% of the dataset")

The appartments that were flagged as invalid in Paris file represent 2.24% of the dataset


## Why dropping the lines?
**EN**  
Dropping lines without further analysis is not a good practice. But for the sake of simplicity, we're going to do it here, since the non-conform appartments represent less than 3 % in all files.  
**FR**  
Supprimer des lignes sans analyse supplémentaire n'est pas une bonne pratique. Mais pour simplifier les choses, nous allons le faire ici, car les appartements non conformes représentent moins de 3 % dans tous les fichiers.


In [7]:
drop_nonconform_lines(paris, 2)
drop_nonconform_lines(lyon, 2)
drop_nonconform_lines(marseille, 2)

prop_printer(paris, 'Paris', 2)
prop_printer(lyon, 'Lyon', 2)
prop_printer(marseille, 'Marseille', 2)

Proportion of flagged lines for Paris: 0.00 %
Proportion of flagged lines for Lyon: 0.00 %
Proportion of flagged lines for Marseille: 0.00 %


In [8]:
df_paris = pd.read_csv(paris, encoding='latin-1', header = None, sep = ';')
df_lyon = pd.read_csv(lyon, encoding='latin-1', header = None, sep = ';')
df_marseille = pd.read_csv(marseille, encoding='latin-1', header = None, sep = ';')
df_final = pd.concat([df_paris, df_marseille, df_lyon], ignore_index=True)
df_final = df_final.rename(columns = {0:'type_logement', 1: 'surface', 2: 'prix'})
app_mask = df_final['type_logement'].str.contains('appartement')
df_final = df_final[app_mask]
df_split = df_final.type_logement.str.split(' ', expand = True)
df_split = df_split.rename(columns = {0: 'type_l', 1 : 'ville', 2: 'arr'})
df_final = pd.concat([df_final, df_split], axis = 1)
df = df_final.copy().drop('type_logement', axis=1)
df.head()

,surface,prix,type_l,ville,arr
0,3 pieces 1 chambre 51.84m2,535 000 eur,appartement,Paris,17
1,3 pieces 2 chambres 40.77m2,470 000 eur,appartement,Paris,15
2,2 pieces 1 chambre 39m2,385 000 eur,appartement,Paris,13
3,2 pieces 1 chambre 36.03m2,383 000 eur,appartement,Paris,13
4,2 pieces 1 chambre 40.7m2,399 000 eur,appartement,Paris,10


**EN**  
The next texp would be to split the surface column into 3 separate columns: Rooms, Bedroom, Area.  
**FR**  
On va ensuite séparer la colonne surface en 3: Nombre de pièces, nombre de chambres, surface.



In [9]:
pattern_pieces = r'(\d+)\s*piece[s]?\s*'
pattern_ch = r'(\d+)\s*chambre[s]?\s*'
df['n_pieces'] = df['surface'].str.extract(pattern_pieces, expand = False).astype(float)
df['n_pieces'] = df['n_pieces'].fillna(0)
df['n_chambres'] = df['surface'].str.extract(pattern_ch, expand = False).astype(float)
df['n_chambres'] = df.n_chambres.fillna(0)

In [10]:
df.head()

,surface,prix,type_l,ville,arr,n_pieces,n_chambres
0,3 pieces 1 chambre 51.84m2,535 000 eur,appartement,Paris,17,3.0,1.0
1,3 pieces 2 chambres 40.77m2,470 000 eur,appartement,Paris,15,3.0,2.0
2,2 pieces 1 chambre 39m2,385 000 eur,appartement,Paris,13,2.0,1.0
3,2 pieces 1 chambre 36.03m2,383 000 eur,appartement,Paris,13,2.0,1.0
4,2 pieces 1 chambre 40.7m2,399 000 eur,appartement,Paris,10,2.0,1.0


**EN**  
We used regex to match the pattern `pièces` and the numeric information for this pattern. We did the same for `chambres`  
**FR**  
Nous avons utilisé des expressions régulières pour faire correspondre le motif "pièces" et les informations numériques pour ce motif. Nous avons fait la même chose pour "chambres".


In [11]:
df['surface_m2'] = df['surface'].str.replace(pattern_pieces, '', regex=True)
df['surface_m2'] = df['surface_m2'].str.replace(pattern_ch, '',regex = True)
df['surface_m2'] = df['surface_m2'].str.replace('m2','',regex=True).str.strip()
df['surface_m2'] = pd.to_numeric(df['surface_m2'], errors = 'coerce')

**EN**  
We will use `regex` to extract only the numeric value in the column `surface` for the area in square meters. We then convert the new column `surface_m2` to float and coerce the errors, replacing them with `NaN`.  
**FR**  
Nous allons utiliser `regex` pour extraire uniquement la valeur numérique de la colonne `surface` pour la superficie en mètres carrés. Ensuite, nous convertirons la nouvelle colonne `surface_m2` en `float` et nous coercerons les erreurs en les remplaçant par `NaN`.


In [12]:
nn_rows = df[pd.to_numeric(df['surface_m2'], errors = 'coerce').isna()]
nn_rows

,surface,prix,type_l,ville,arr,n_pieces,n_chambres,surface_m2
11,3 pieces,Prix NC,appartement,Paris,14,3.0,0.0,NaN
16,4 pieces,Prix NC,appartement,Paris,14,4.0,0.0,NaN
21,5 pieces,Prix NC,appartement,Paris,14,5.0,0.0,NaN
68,3/4 pieces 2 chambres 75.24m2,670 000 eur,appartement,Paris,17,4.0,2.0,NaN
178,5 pieces,Prix NC,appartement,Paris,15,5.0,0.0,NaN
192,2/3 pieces 2 chambres 54m2,715 000 eur,appartement,Paris,16,3.0,2.0,NaN
196,3/4 pieces 2 chambres 102m2,1 235 000 eur,appartement,Paris,16,4.0,2.0,NaN
272,2/3 pieces 1 chambre 48.36m2,610 000 eur,appartement,Paris,15,3.0,1.0,NaN
384,4 pieces,Prix NC,appartement,Paris,15,4.0,0.0,NaN
389,2 pieces,Prix NC,appartement,Paris,15,2.0,0.0,NaN


In [13]:
df['surface_m2'].isna().sum()

46

**EN**  
We look further into the NaN values and their number. A lot of them have NaN values in both the `prix` and `surface_m2` columns. We can safely drop the rows.  
**FR**    
Nous examinons davantage les valeurs `NaN` et leur nombre. Beaucoup d'entre elles ont des valeurs `NaN` dans les deux colonnes `prix` et `surface_m2`.


In [14]:
df['prix'] = df['prix'].str.replace('eur| ', '', regex=True).str.strip()
df['prix'] = pd.to_numeric(df['prix'], errors='coerce')
df = df.dropna(subset=['prix', 'surface_m2'])
cols = ["type_l", "ville", "arr", "n_pieces", "n_chambres", "surface_m2", "prix"]
df_pml = df.loc[:, cols]
df_pml['prix_m2'] = df['prix'] // df['surface_m2']
df_pml['arr'] = df_pml.arr.astype('int')
df_pml.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5973 entries, 0 to 6486
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type_l      5973 non-null   object 
 1   ville       5973 non-null   object 
 2   arr         5973 non-null   int32  
 3   n_pieces    5973 non-null   float64
 4   n_chambres  5973 non-null   float64
 5   surface_m2  5973 non-null   float64
 6   prix        5973 non-null   float64
 7   prix_m2     5973 non-null   float64
dtypes: float64(5), int32(1), object(2)
memory usage: 396.6+ KB


**EN**  
Finally, we cleaned and converted the price column. We will now save our dataframe as a csv file and proceed to Exploratory Data Analysis. We will create for now a column `price_m2` that will be used only for `EDA` but we'll make sure to drop it before proceeding to any Machine Learning algorithm, for obvious reasons.  
**FR**  
Enfin, nous avons nettoyé et converti la colonne des prix. Nous allons maintenant enregistrer notre dataframe sous forme de fichier csv et procéder à l'analyse exploratoire des données. Nous allons créer pour le moment une colonne `prix_m2` qui sera utilisée uniquement pour l'`AED`, mais nous veillerons à la supprimer avant de procéder à un algorithme d'apprentissage automatique, pour des raisons évidentes.

In [15]:
#cwd = os.getcwd()
#filename = "real_estate_PML.csv"
#filepath = os.path.join(cwd, filename)
#df_pml.to_csv(filepath, index = False)

In [16]:
df_pml.head(10)

,type_l,ville,arr,n_pieces,n_chambres,surface_m2,prix,prix_m2
0,appartement,Paris,17,3.0,1.0,51.84,535000.0,10320.0
1,appartement,Paris,15,3.0,2.0,40.77,470000.0,11528.0
2,appartement,Paris,13,2.0,1.0,39.00,385000.0,9871.0
3,appartement,Paris,13,2.0,1.0,36.03,383000.0,10630.0
4,appartement,Paris,10,2.0,1.0,40.70,399000.0,9803.0
5,appartement,Paris,15,2.0,1.0,41.22,560000.0,13585.0
6,appartement,Paris,19,3.0,1.0,51.05,428000.0,8383.0
7,appartement,Paris,16,2.0,0.0,20.93,185700.0,8872.0
8,appartement,Paris,19,2.0,1.0,46.80,449000.0,9594.0
9,appartement,Paris,10,3.0,2.0,50.30,530000.0,10536.0
